In [2]:
from tidy_data import get_long_education
import pandas as pd
pd.set_option('display.max_rows', 500)
import numpy as np

import json
import re
import string

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [3]:
with open('data/raw/scraped_profiles.json') as json_data:
    profiles = json_data.readlines()

In [4]:
educ = get_long_education(profiles); print(len(educ))
educ.loc[educ.discipline == '','discipline'] = educ.loc[educ.discipline == '','degree']
educ.id.nunique()

40735


18531

## Degree
1. Take degree abbreviations
2. Find keywords in degree/discipline fields

**Get degree abbreviations**

In [5]:
### List of degrees abbreviations from https://en.wikipedia.org/wiki/British_degree_abbreviations
ab = pd.read_csv('data/degrees_ abbreviations.csv', sep=';')
ab['ab'] = ab['ab'].apply(lambda x: "\\b" + x.lower().strip() + "\\b")

In [6]:
bach_ab = "|".join(ab[ab.degree=='Bachelor'].ab.values) + "|bachelor|\\bbsc\\b|bachelors|\\bba\\b|\\bbba\\b|\\bbss\\b|\\bbac\\b|\\bb sc\\b|\
                  \\bbtech\\b|\\bbasc\\b|\\bbcomm\\b|\\bbcs\\b|\\bbmath\\b|\\bbe\\b|\\bbtech\\b|undergraduate|licencia|\\bbs\\b"
mas_ab = "|".join(ab[ab.degree=='Master'].ab.values) + "|master|\\bmsc\\b|masters|\\bmss\\b|\\bmphil\\b|magister|\\bmasc\\b|\\bmtech\\b|\\bma\\b|\\bms\\b"
phd_ab = "|".join(ab[ab.degree=='PHD'].ab.values) + "|\\bph\\b|doctorate|doctor|\\bdrs\\b|\\bdr\\b"

In [7]:
clean_titles = ["high school", 'bachelor', 'master', 'phd', 'diploma','certificate',"postdoc"]
clean_titles = "|".join(clean_titles)

**Preprocess and find abbreviations keywords**

In [8]:
def process_degree(degree):
    text = degree.lower().strip()
    try:
        text2 = list(set(re.findall(clean_titles, text)))
        if len(text2) > 0:
            text = " ".join(text2)
    except:
        pass
    
    if text.startswith('b.') or text.startswith('b-') or text.startswith('b '):
        return 'bachelor'
    elif text.startswith('m.') or text.startswith('m-') or text.startswith('m '):
        return 'master'
    elif text.startswith('d.') or text.startswith('d-'):
        return 'phd'
    
    remove_punct = str.maketrans(string.punctuation, ' '*len(string.punctuation))
    text = text.translate(remove_punct)
    
    remove_digits = str.maketrans("", "", string.digits)
    text = text.translate(remove_digits)
    
    if text.startswith('bs') or text.startswith('b '):
        return 'bachelor'
    elif text.startswith('ms') or text.startswith('m '):
        return 'master'
    
    text = text.replace(' ml ', 'machine learning').replace(' ai ', 'artificial intelligence')

    text = re.sub(phd_ab,"phd",text)
    text = re.sub(mas_ab,"master",text)
    text = re.sub(bach_ab,"bachelor",text)    
    text = re.sub("diplom|\\bdipl\\b","diploma",text)    
    text = re.sub("\\bcertificat\\b|certification","certificate",text)
    text = re.sub('secondary school|higher secondary|highschool|high school',"high school",text)

    
    if 'phd' in text:
        return 'phd'
    elif 'master' in text:
        return 'master'
    elif 'bachelor' in text:
        return 'bachelor'
    else:
        try:
            text2 = list(set(re.findall(titles, text)))
            if len(text2) > 0:
                text = text2[0]
            else:
                text3 = list(set(re.findall("honour[s]?|laude|\\bdegree\\b", text)))
                if len(text3) > 0:
                    text = "honours"
        except:
            pass
    return text

In [9]:
### Sometimes people confuse the 'degree' and 'discipline' fields on LinkedIn.
### Clean both, and then unify them
educ['degree2'] = educ.degree.apply(lambda x: process_degree(x))
educ['degree3'] = educ.discipline.apply(lambda x: process_degree(x))

In [10]:
def final_class(degree2, degree3, duration):
    degrees = []
    for i in range(len(degree2)):
        raw_degrees = degree2[i].split() + degree3[i].split()
        if 'postdoc' in raw_degrees:
            text = 'postdoc'
        elif 'phd' in raw_degrees:
            text = 'phd'
        elif 'master' in raw_degrees:
            text = 'master'
        elif 'bachelor' in raw_degrees:
            text = 'bachelor'
        elif 'high school' in raw_degrees:
            text = 'high school'
        else:
            if duration[i] >= 3:
                text = 'bachelor'
            else:
                text = 'certificate'

        degrees.append(text)
                    
    return degrees

In [11]:
educ['final_degree'] = final_class(educ.degree2.values,educ.degree3.values, educ.duration.values)

In [12]:
educ.final_degree.value_counts()

bachelor       16310
master         11636
certificate    10361
phd             2397
postdoc           31
Name: final_degree, dtype: int64

In [13]:
#educ.to_csv('data/processed/educ_df_v2.csv',index=False)

# Matching to CAH categories
1. Remove degree abbreviations from discipline field and pre-process
2. Match to CAH categories

**Preprocess discipline fields and remove degree keywords**

In [14]:
ab_titles = ab.ab.values
ab_titles = "|".join(ab_titles)

def process_discipline(discipline):
    text = discipline.lower()
    # Taken from stackoverflow. replace punctuation with spaces
    text = (text.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation))).
            replace(' '*4, ' ').
            replace(' '*3, ' ').
            replace(' '*2, ' ').strip())
    
    remove_digits = str.maketrans("", "", string.digits)
    text = text.translate(remove_digits)

    # remove degree titles
    text = re.sub(ab_titles,"",text)
    stop_words = stopwords.words('english') + ['general','executive','diploma','applied','degree',\
                                              'distinction','masters','master','bachelors','bachelor']
    stop_words.remove('it')
    
    tokens = word_tokenize(text)
    tokens_stop = [y for y in tokens if y not in stop_words]
    tokens.sort()
    text = " ".join(tokens_stop).strip()

    return text

In [15]:
educ['disc2'] = educ.discipline.apply(lambda x: process_discipline(x));
educ.loc[educ.disc2 == '','disc2'] = educ.loc[educ.disc2 == '','degree2']

In [ ]:
disc_counts = educ.disc2.value_counts().reset_index(); print(len(disc_counts))
disc_counts.columns = ['disc2','counts']

**Create word vectors with CAH categories**

In [16]:
cah = pd.read_csv('data/CAH_categories.csv', delimiter=";")
cah['CAH3'] = cah['CAH3'].apply(lambda x: re.sub("[\(\[].*?[\)\]]", "",x).strip()) ### remove text in ()

In [17]:
# Add missing categories
cah = cah.append(pd.DataFrame({
    "CAH1": ['(CAH17) business and management'],
    "CAH2": ['(CAH17-01) business and management'],
    "CAH3": ['mba']
}))

In [21]:
nlp = spacy.load("en_core_web_lg")

In [22]:
# Create word vectors with CAH categories to use as reference
all_base_categories = list(cah.CAH3.values)
print('creating base vectors... ', end = '')
base_vectors = [nlp(x) for x in all_base_categories]
print('done!')

creating base vectors... done!


**Create word vectors with clean disciplines from LinkedIn**

In [23]:
# Create word vectors with clean disciplines to be matched
vector_dict = {}
for disc2 in educ[~educ.disc2.isna()].disc2.unique():
    vector_dict[disc2] = nlp(disc2)
    
raw_title = []
tokens = []
for key,value in vector_dict.items():
    raw_title.append(key)
    tokens.append(value)
    
raw_disc_df = pd.DataFrame({'raw':raw_title, 'token':tokens})

**Match disciplines to CAH**

In [26]:
def get_most_similar_title(raw_occupations, base_vectors, base_occupations):
    most_similar_title = []
    similarity_score = []
    count = 0
    for raw_vector in raw_occupations.token.values: 
        this_score = []
        for base_vector in base_vectors:
            this_score.append(raw_vector.similarity(base_vector))
        similarity_score.append(max(this_score))
        most_similar_title.append(base_occupations[this_score.index(max(this_score))])
        count += 1
    return pd.DataFrame({"disc2": raw_disc_df.raw.values, "CAH3": most_similar_title, "score": similarity_score})

In [27]:
matched_df = get_most_similar_title(raw_disc_df, base_vectors, all_base_categories)
matched_df = matched_df.merge(cah[['CAH1','CAH3']], on='CAH3',how='left')
matched_df.loc[matched_df.score<0.5,'CAH1'] = ""
matched_df.loc[matched_df.score<0.5,'CAH3'] = ""

<ipython-input-26-9bc1b7ceed59>:8: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  this_score.append(raw_vector.similarity(base_vector))


In [28]:
len(matched_df)

14677

In [29]:
educ2 = educ.merge(matched_df, on='disc2',how='left')

In [30]:
educ2.CAH1.value_counts()

(CAH17) business and management                            8825
                                                           7965
(CAH11) computing                                          7917
(CAH10) engineering and technology                         5307
(CAH15) social sciences                                    2457
(CAH07) physical sciences                                  1579
(CAH22) education and teaching                             1450
(CAH09) mathematical sciences                              1183
(CAH19) language and area studies                           767
(CAH02) subjects allied to medicine                         709
(CAH03) biological and sport sciences                       449
(CAH25) design, and creative and performing arts            318
(CAH23) combined and general studies                        317
(CAH24) media, journalism and communications                235
(CAH16) law                                                 232
(CAH04) psychology                      

**Get maximum level of education**

In [31]:
def get_max_degree(degrees):
    degrees = list(degrees.values)
    if degrees == "":
        return 'not specified'
    elif 'postdoc' in degrees:
        return 'postdoc'
    elif 'phd' in degrees:
        return 'phd'
    elif 'master' in degrees:
        return 'master'
    elif 'bachelor' in degrees:
        return 'bachelor'
    else:
        return 'not university'

In [32]:
max_degree = educ.groupby('id')['final_degree'].apply(lambda x: get_max_degree(x)).reset_index().rename(columns={'final_degree':'max_degree'})

In [33]:
educ.final_degree.value_counts()

bachelor       16310
master         11636
certificate    10361
phd             2397
postdoc           31
Name: final_degree, dtype: int64

In [34]:
max_degree.max_degree.value_counts()

master            8192
bachelor          7081
phd               2261
not university     969
postdoc             28
Name: max_degree, dtype: int64

In [36]:
### Add years spent studying
educ2 = educ2.merge(max_degree, on='id',how='left').merge(educ2.groupby('id').duration.sum().reset_index().rename(columns={'duration':"years_education"}), on='id',how='left')


In [37]:
# educ2.drop(['name','debug','degree2','degree3','disc2'],axis=1).to_csv('data/processed/educ_df_3.csv',index=False)